In [1]:
%cd ../
import os
import sys
import time
import requests
import pandas as pd
from scipy import signal
from scipy import fftpack

import numpy as np
from matplotlib import pyplot as plt
from stix.core import stix_datatypes as sdt
from stix.core import mongo_db as db
from stix.core import stix_datetime
import pymongo
from dateutil import parser as dtparser

connect = pymongo.MongoClient()
db = connect["stix"]
%matplotlib notebook
mdb=db.MongoDB()
default_folder = '/data/flare_search/'
SPID = 54118

/home/xiaohl/FHNW/STIX/gsw/STIX_python
[INFO 2020-09-11 11:23:35] : IDB loaded from stix/data/idb/idb.sqlite


TypeError: 'Collection' object is not callable. If you meant to call the 'MongoDB' method on a 'Database' object it is failing because no such method exists.

## 1.  Data preparation

In [ ]:


def get_lc_data(file_id):
    # query database and retrieve the packets
    
    packets = mdb.select_packets_by_run(file_id, SPID)
    if not packets:
        print(f'No QL LC packets found for run {file_id}')
        return None, None
    else:
        print('Number of  LC packets found for run {}: {}'.format(file_id,packets.count()))

    lightcurves = {}
    unix_time = []
    for pkt in packets:
        packet = sdt.Packet(pkt)
        if not packet.isa(SPID):
            continue
        #fig = None

        scet_coarse = packet[1].raw
        scet_fine = packet[2].raw
        start_scet = scet_coarse + scet_fine / 65536.

        int_duration = (packet[3].raw + 1) * 0.1

        detector_mask = packet[4].raw
        pixel_mask = packet[6].raw

        num_lc = packet[17].raw

        compression_s = packet[8].raw
        compression_k = packet[9].raw
        compression_m = packet[10].raw

        num_lc_points = packet.get('NIX00270/NIX00271')[0]
        lc = packet.get('NIX00270/NIX00271/*.eng')[0]
        rcr = packet.get('NIX00275/*.raw')
        UTC = packet['header']['UTC']
        for i in range(len(lc)):
            if i not in lightcurves:
                lightcurves[i] = []
            lightcurves[i].extend(lc[i])
        unix_time.extend([
            stix_datetime.scet2unix(start_scet + x * int_duration)
            for x in range(num_lc_points[0])
        ])
    return lightcurves, unix_time



In [ ]:
lcs,t=get_lc_data(268)
lightcurve=np.array(lcs[0]) #4-10 keV
unix_time=np.array(t)

mean_counts=np.mean(lightcurve)

std_counts=np.std(lightcurve)
print(mean_counts,std_counts)


## 2. Peak detection algorithm

In [ ]:
#peak detection algorithm


def find_peaks(data,peak_width=10, distance=75):
    mean_counts=np.mean(data)
    std_counts=20 #np.std(data)
    height=mean_counts+std_counts
    prominence=std_counts
    #find local maximums which satisfy the conditions
    
    
    xpeak, properties = signal.find_peaks(data, 
                                          height=height,
                                         # threshold=threshold,
                                          prominence=prominence,
                                                width=peak_width,
                                                distance=distance
                                         )
    print('Number of detected peaks:', xpeak.size)
    #print(xpeak.size)



    if xpeak.size>0:
        peak_values=data[xpeak]
        peak_unix_times = unix_time[xpeak]
        peaks_utc=[stix_datetime.unix2utc(x) for x in peak_unix_times]

        fig1=plt.figure()

        plt.plot(unix_time, data)
        plt.yscale('log')
        plt.vlines(peak_unix_times, 0.8*peak_values, 1.5*peak_values,color='C1')
        plt.show()

        #fig2=plt.figure()


        #N=len(data)
        #fs=0.25
        #time = np.arange(N) / float(fs)


        #f,t,Zxx = signal.stft(data, fs=0.25, nperseg=200)
        #amp = 2 * np.sqrt(2)
        #plt.pcolormesh(t, f, np.abs(Zxx), vmin=0, vmax=amp, shading='gouraud')
        #plt.title('STFT Magnitude')
        #plt.xlabel('Time [s]')
        #plt.ylabel('Frequency (Hz)')
        #plt.show()
        return peak_unix_times, peak_values


t, v=find_peaks(lightcurve)



### 2.1 Low-pass filter

In [ ]:
# Low pass filter to fiter high frequence pattern caused  by integer compression, noise etc.

Wn=0.03 #cut off frequency    

order=4 #butterworth filter order
b,a=signal.butter(order, Wn, 'low', analog=False)#analog 
w, h = signal.freqs(b, a)
fig=plt.figure()
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [1 / second]')
#plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
#plt.axvline(Wn, color='green') # cutoff frequency
plt.show()
    
#y = signal.filtfilt(b, a, data)




In [ ]:
lc_smoothed = signal.filtfilt(b, a, lightcurve)
fig=plt.figure()
plt.plot(unix_time,lightcurve,'r', label='orignal')
plt.plot(unix_time,lc_smoothed,'b', label='smoothed')
plt.yscale('log')
plt.legend(loc='upper left')
plt.show()

In [ ]:
times, peak_values= find_peaks(lc_smoothed)

In [ ]:
fig3=plt.figure()
plt.plot(unix_time, lightcurve)
plt.vlines(times, 0.8*peak_values, 1.5*peak_values,color='C1')
plt.show()

In [ ]:
width=75
order=2
lc_savgol=signal.savgol_filter(lightcurve, width, order)

In [ ]:
fig4=plt.figure()
plt.plot(unix_time, lightcurve,'b', label='original')
plt.plot(unix_time,lc_savgol,'r',label='smoothed')
#plt.plot(unix_time,lc_smoothed,'g',label='smoothed 2')
plt.legend('upper right')
plt.show()

In [ ]:
times_savgol, peak_values_savgol= find_peaks(lc_savgol)

In [ ]:
fig=plt.figure()
plt.plot(times,peak_values,'x')
plt.vlines(times_savgol, 0.8*peak_values_savgol,peak_values_savgol*1.2, color='C1')
plt.show()

In [ ]:
np.std(lightcurve[0:500])